In [40]:
#Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


# Pre-Processing: Data Cleaning & Feature Engineering

In [41]:
df = pd.read_csv("..\\..\\data\\raw\\CarSellDataset.csv")

In [42]:
display(df.head(10))

,Brand,Model Name,Model Variant,Car Type,Transmission,Fuel Type,Year,Kilometers,Owner,State,Accidental,Price
0,Mahindra,TUV300,AX5,SUV,Manual,CNG,2017,164654,1st,Rajasthan,No,547253
1,Skoda,Rapid,Style,Sedan,Manual,Petrol,2018,41351,1st,Maharashtra,No,512050
2,Maruti Suzuki,Alto,Z,Hatchback,Manual,Diesel,2002,119090,3rd+,Tamil Nadu,No,678923
3,Hyundai,Grand i10,Magna,Hatchback,Manual,Diesel,2013,19979,1st,Andhra Pradesh,No,522500
4,Mahindra,XUV500,W8,SUV,Manual,Petrol,2011,130591,3rd+,Bihar,No,401182
5,Maruti Suzuki,Dzire,Base,Sedan,Manual,Diesel,2019,71192,1st,Uttar Pradesh,No,219848
6,MG,Hector,Smart,SUV,Manual,CNG,2023,61255,2nd,Maharashtra,No,1092070
7,Maruti Suzuki,Baleno,L,Hatchback,Manual,Diesel,2017,55072,1st,Gujarat,No,643104
8,Maruti Suzuki,Ignis,L,Hatchback,Manual,Electric,2022,130490,1st,Bihar,No,1128600
9,Maruti Suzuki,Swift,V,Sedan,Automatic,Petrol,2018,103459,1st,Gujarat,No,181046


In [43]:
#Shape of the dataset
display(df.shape)

(140904, 12)

In [44]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140904 entries, 0 to 140903
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Brand          140904 non-null  object
 1   Model Name     140904 non-null  object
 2   Model Variant  140904 non-null  object
 3   Car Type       140904 non-null  object
 4   Transmission   140904 non-null  object
 5   Fuel Type      140904 non-null  object
 6   Year           140904 non-null  int64 
 7   Kilometers     140904 non-null  int64 
 8   Owner          140904 non-null  object
 9   State          140904 non-null  object
 10  Accidental     140904 non-null  object
 11  Price          140904 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 12.9+ MB


None

In [45]:
df.isna().sum() #No of Null values in each columns

Brand            0
Model Name       0
Model Variant    0
Car Type         0
Transmission     0
Fuel Type        0
Year             0
Kilometers       0
Owner            0
State            0
Accidental       0
Price            0
dtype: int64

In [46]:
print(df.nunique()) #No of Unique values in each column

Brand               18
Model Name          84
Model Variant       82
Car Type             5
Transmission         2
Fuel Type            5
Year                24
Kilometers       95860
Owner                3
State               27
Accidental           2
Price            17745
dtype: int64


In [47]:
df["Brand"].unique()    #Unique values in Brand column

array(['Mahindra', 'Skoda', 'Maruti Suzuki', 'Hyundai', 'MG', 'Audi',
       'Toyota', 'Honda', 'Tata', 'Ford', 'Chevrolet', 'BMW',
       'Volkswagen', 'Jaguar', 'Renault', 'Kia', 'Range Rover', 'Nissan'],
      dtype=object)

In [48]:
#unique values in each column
for col in df.columns:
    print(f"Distinct values in column '{col}':")
    display(df[col].unique())
    print("\n" + "="*50 + "\n") # Separator

Distinct values in column 'Brand':


array(['Mahindra', 'Skoda', 'Maruti Suzuki', 'Hyundai', 'MG', 'Audi',
       'Toyota', 'Honda', 'Tata', 'Ford', 'Chevrolet', 'BMW',
       'Volkswagen', 'Jaguar', 'Renault', 'Kia', 'Range Rover', 'Nissan'],
      dtype=object)



Distinct values in column 'Model Name':


array(['TUV300', 'Rapid', 'Alto', 'Grand i10', 'XUV500', 'Dzire',
       'Hector', 'Baleno', 'Ignis', 'Swift', 'Ertiga', 'A6',
       'Urban Cruiser', 'City', 'Verna', 'Celerio', 'Jazz', 'Brio',
       'Harrier', 'Venue', 'Aspire', 'Tavera', 'Innova', 'Altroz',
       'WagonR', 'Aura', 'Creta', 'Figo', '5 Series', 'Ameo', 'XE',
       'Nexon', 'Vento', 'Kwid', 'i20', 'WR-V', 'Tiago', 'Kiger',
       'EcoSport', 'Sonet', 'Tigor', 'S-Presso', 'Bolero', 'Endeavour',
       'Beat', 'Amaze', 'Taigun', 'Glanza', 'Seltos', 'Fortuner', 'Thar',
       'Polo', 'Q7', 'Superb', 'X3', 'Evoque', 'A4', 'Triber', 'i10',
       'Slavia', 'Etios', 'F-Pace', 'ZS EV', 'Scorpio', 'Cruze', 'Carens',
       'Terrano', 'Astor', 'Duster', 'Sunny', 'Kushaq', 'Octavia',
       'Micra', '3 Series', 'Velar', 'X1', 'Magnite', 'Discovery Sport',
       'XF', 'Sail', 'Q5', 'Q3', 'Spark', 'Sport'], dtype=object)



Distinct values in column 'Model Variant':


...

array(['Rajasthan', 'Maharashtra', 'Tamil Nadu', 'Andhra Pradesh',
       'Bihar', 'Uttar Pradesh', 'Gujarat', 'West Bengal', 'Kerala',
       'Madhya Pradesh', 'Haryana', 'Telangana', 'Delhi', 'Punjab',
       'North East', 'Karnataka', 'Jharkhand', 'Goa', 'Odisha',
       'Himachal Pradesh', 'Chhattisgarh', 'Uttarakhand', 'Puducherry',
       'Assam', 'Jammu & Kashmir', 'Chandigarh', 'Other UTs'],
      dtype=object)



Distinct values in column 'Accidental':


array(['No', 'Yes'], dtype=object)



Distinct values in column 'Price':


array([ 547253,  512050,  678923, ...,  493143,  192727, 1070129],
      shape=(17745,))

In [49]:
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()  #standardize missing values, removing space and adding underscore

df = df.drop_duplicates() #drop the duplicate enteries

df = df.dropna()

display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140904 entries, 0 to 140903
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   brand          140904 non-null  object
 1   model_name     140904 non-null  object
 2   model_variant  140904 non-null  object
 3   car_type       140904 non-null  object
 4   transmission   140904 non-null  object
 5   fuel_type      140904 non-null  object
 6   year           140904 non-null  int64 
 7   kilometers     140904 non-null  int64 
 8   owner          140904 non-null  object
 9   state          140904 non-null  object
 10  accidental     140904 non-null  object
 11  price          140904 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 12.9+ MB


None

In [50]:
for col in ["price","kilometers"]:
  if col in df.columns:
    df[col] = df[col].astype(str).str.replace(',', '').str.replace('₹', '').astype(float)

print("\nCleaned dataset info:")
df.info()


Cleaned dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140904 entries, 0 to 140903
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   brand          140904 non-null  object 
 1   model_name     140904 non-null  object 
 2   model_variant  140904 non-null  object 
 3   car_type       140904 non-null  object 
 4   transmission   140904 non-null  object 
 5   fuel_type      140904 non-null  object 
 6   year           140904 non-null  int64  
 7   kilometers     140904 non-null  float64
 8   owner          140904 non-null  object 
 9   state          140904 non-null  object 
 10  accidental     140904 non-null  object 
 11  price          140904 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 12.9+ MB


In [51]:
df.head()

,brand,model_name,model_variant,car_type,transmission,fuel_type,year,kilometers,owner,state,accidental,price
0,Mahindra,TUV300,AX5,SUV,Manual,CNG,2017,164654.0,1st,Rajasthan,No,547253.0
1,Skoda,Rapid,Style,Sedan,Manual,Petrol,2018,41351.0,1st,Maharashtra,No,512050.0
2,Maruti Suzuki,Alto,Z,Hatchback,Manual,Diesel,2002,119090.0,3rd+,Tamil Nadu,No,678923.0
3,Hyundai,Grand i10,Magna,Hatchback,Manual,Diesel,2013,19979.0,1st,Andhra Pradesh,No,522500.0
4,Mahindra,XUV500,W8,SUV,Manual,Petrol,2011,130591.0,3rd+,Bihar,No,401182.0


In [52]:
df.describe()

,year,kilometers,price
count,140904.000000,140904.000000,1.409040e+05
mean,2016.960391,95024.595987,7.617872e+05
std,5.106106,49133.157878,4.438578e+05
min,2000.000000,10000.000000,5.005500e+04
25%,2014.000000,52421.000000,4.116420e+05
50%,2018.000000,94973.500000,6.828030e+05
75%,2021.000000,137618.000000,1.034178e+06
max,2023.000000,179998.000000,2.744280e+06


In [53]:
display(df['brand'].value_counts()) #each brand counts

brand
Maruti Suzuki    54030
Hyundai          22090
Honda            11936
Toyota           10483
Mahindra          9618
Tata              8825
Ford              5217
Volkswagen        3716
Renault           3024
Nissan            2184
Kia               2146
Skoda             1537
MG                1492
BMW               1052
Audi              1012
Range Rover        906
Jaguar             884
Chevrolet          752
Name: count, dtype: int64

In [54]:
import datetime

current_year = datetime.datetime.now().year
df['car_age'] = current_year - df['year']

df['price_per_km'] = df['price'] / df['kilometers']
df['price_per_km'].replace([float('inf'), -float('inf')], 0, inplace=True)

C:\Users\SU20171413\AppData\Local\Temp\ipykernel_17984\693759571.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price_per_km'].replace([float('inf'), -float('inf')], 0, inplace=True)


In [55]:
df.head()

,brand,model_name,model_variant,car_type,transmission,fuel_type,year,kilometers,owner,state,accidental,price,car_age,price_per_km
0,Mahindra,TUV300,AX5,SUV,Manual,CNG,2017,164654.0,1st,Rajasthan,No,547253.0,8,3.323654
1,Skoda,Rapid,Style,Sedan,Manual,Petrol,2018,41351.0,1st,Maharashtra,No,512050.0,7,12.383014
2,Maruti Suzuki,Alto,Z,Hatchback,Manual,Diesel,2002,119090.0,3rd+,Tamil Nadu,No,678923.0,23,5.700924
3,Hyundai,Grand i10,Magna,Hatchback,Manual,Diesel,2013,19979.0,1st,Andhra Pradesh,No,522500.0,12,26.152460
4,Mahindra,XUV500,W8,SUV,Manual,Petrol,2011,130591.0,3rd+,Bihar,No,401182.0,14,3.072049


In [56]:
# Save DataFrame to CSV
df.to_csv("..\\..\\data\\intermediate\\firstCarDataset.csv", index=False)